[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/semantic-router/blob/main/docs/encoders/openai-embed-3.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/aurelio-labs/semantic-router/blob/main/docs/encoders/openai-embed-3.ipynb)

# Using Cohere Embedding Models

## Getting Started

We start by installing semantic-router, as of version `0.1.8` the `CohereEncoder` became available via the base installation (ie no need to install via `semantic-router[cohere]`).

In [ ]:
!pip install -qU "semantic-router>=0.1.8"

We start by defining a dictionary mapping routes to example phrases that should trigger those routes.

In [1]:
from semantic_router import Route

politics = Route(
    name="politics",
    utterances=[
        "isn't politics the best thing ever",
        "why don't you tell me about your political opinions",
        "don't you just love the president",
        "don't you just hate the president",
        "they're going to destroy this country!",
        "they will save the country!",
    ],
)

/Users/jamesbriggs/Documents/aurelio/semantic-router/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-07 17:04:01 - httpx - INFO - _client.py:1013 - _send_single_request() - HTTP Request: GET https://raw.githubusercontent.com/BerriAI/litellm/main/model_prices_and_context_window.json "HTTP/1.1 200 OK"


Let's define another for good measure:

In [2]:
chitchat = Route(
    name="chitchat",
    utterances=[
        "how's the weather today?",
        "how are things going?",
        "lovely weather today",
        "the weather is horrendous",
        "let's go to the chippy",
    ],
)

routes = [politics, chitchat]

Now we initialize our embedding model, we will use the `-3-large` model alongside a `dimensions` value of `256`. This will produce _tiny_ 256-dimensional vectors that — according to OpenAI — outperform the 1536-dimensional vectors produced by `text-embedding-ada-002`.

In [3]:
import os
from getpass import getpass
from semantic_router.encoders import CohereEncoder

os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY") or getpass(
    "Enter Cohere API Key: "
)

encoder = CohereEncoder(name="embed-english-v3.0", score_threshold=0.3)

Now we define the `RouteLayer`. When called, the route layer will consume text (a query) and output the category (`Route`) it belongs to — to initialize a `RouteLayer` we need our `encoder` model and a list of `routes`.

In [4]:
from semantic_router.routers import SemanticRouter

rl = SemanticRouter(encoder=encoder, routes=routes, auto_sync="local")

2025-04-07 17:04:21 - semantic_router.utils.logger - WARNING - base.py:442 - _get_index() - No index provided. Using default LocalIndex.
2025-04-07 17:04:21 - httpx - INFO - _client.py:1013 - _send_single_request() - HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
17:04:21 - LiteLLM:INFO: utils.py:1177 - Wrapper: Completed Call, calling success_handler
2025-04-07 17:04:21 - LiteLLM - INFO - utils.py:1177 - wrapper() - Wrapper: Completed Call, calling success_handler
17:04:21 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: cohere/embed-english-v3.0
2025-04-07 17:04:21 - LiteLLM - INFO - cost_calculator.py:622 - completion_cost() - selected model name for cost calculation: cohere/embed-english-v3.0
17:04:21 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: cohere/embed-english-v3.0
2025-04-07 17:04:21 - LiteLLM - INFO - cost_calculator.py:622 - completion_cost() - selected model name for cost calculat

We can check the dimensionality of our vectors by looking at the `index` attribute of the `RouteLayer`.

In [5]:
rl.index.dimensions

1024

Now let's test our router with cohere:

In [6]:
rl("don't you love politics?")

2025-04-07 17:04:35 - httpx - INFO - _client.py:1013 - _send_single_request() - HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
17:04:35 - LiteLLM:INFO: utils.py:1177 - Wrapper: Completed Call, calling success_handler
2025-04-07 17:04:35 - LiteLLM - INFO - utils.py:1177 - wrapper() - Wrapper: Completed Call, calling success_handler
17:04:35 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: cohere/embed-english-v3.0
2025-04-07 17:04:35 - LiteLLM - INFO - cost_calculator.py:622 - completion_cost() - selected model name for cost calculation: cohere/embed-english-v3.0


17:04:35 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: cohere/embed-english-v3.0
2025-04-07 17:04:35 - LiteLLM - INFO - cost_calculator.py:622 - completion_cost() - selected model name for cost calculation: cohere/embed-english-v3.0
17:04:37 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: cohere/embed-english-v3.0
2025-04-07 17:04:37 - LiteLLM - INFO - cost_calculator.py:622 - completion_cost() - selected model name for cost calculation: cohere/embed-english-v3.0
17:04:40 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: cohere/embed-english-v3.0
2025-04-07 17:04:40 - LiteLLM - INFO - cost_calculator.py:622 - completion_cost() - selected model name for cost calculation: cohere/embed-english-v3.0


RouteChoice(name='politics', function_call=None, similarity_score=None)

In [7]:
rl("how's the weather today?")

2025-04-07 17:04:37 - httpx - INFO - _client.py:1013 - _send_single_request() - HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
17:04:37 - LiteLLM:INFO: utils.py:1177 - Wrapper: Completed Call, calling success_handler
2025-04-07 17:04:37 - LiteLLM - INFO - utils.py:1177 - wrapper() - Wrapper: Completed Call, calling success_handler
17:04:37 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: cohere/embed-english-v3.0
2025-04-07 17:04:37 - LiteLLM - INFO - cost_calculator.py:622 - completion_cost() - selected model name for cost calculation: cohere/embed-english-v3.0


RouteChoice(name='chitchat', function_call=None, similarity_score=None)

Both are classified accurately, what if we send a query that is unrelated to our existing `Route` objects?

In [8]:
rl("I'm interested in learning about llama 2")

2025-04-07 17:04:40 - httpx - INFO - _client.py:1013 - _send_single_request() - HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
17:04:40 - LiteLLM:INFO: utils.py:1177 - Wrapper: Completed Call, calling success_handler
2025-04-07 17:04:40 - LiteLLM - INFO - utils.py:1177 - wrapper() - Wrapper: Completed Call, calling success_handler
17:04:40 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: cohere/embed-english-v3.0
2025-04-07 17:04:40 - LiteLLM - INFO - cost_calculator.py:622 - completion_cost() - selected model name for cost calculation: cohere/embed-english-v3.0


RouteChoice(name=None, function_call=None, similarity_score=None)

In this case, we return `None` because no matches were identified. We always recommend optimizing your `RouteLayer` for optimal performance, you can see how in [this notebook](https://github.com/aurelio-labs/semantic-router/blob/main/docs/06-threshold-optimization.ipynb).

---